# Technical Indicators

## 0. Imports & Basic formating

### a. Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam

In [36]:
def import_data(path):
    #############################
    #  1 - Import data          #
    #############################
    data = pd.read_csv(path)
    df = data.copy()

    return df

def import_data_sent(path, columns):
    #############################
    #  1 - Make copy of df      #
    #############################
    df = import_data(path)
    sentimental_data = df[columns]
    sent_df = sentimental_data.copy()

    return sent_df

### b. Basic Formating

In [3]:
def df_formating(formated_df, formated_df_columns):
    """
    Preprocess a DataFrame by renaming columns, setting columns to float64,
    dropping unnecessary columns, setting the 'date' column to datetime type,
    and setting the 'date' column as the index.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - define which columns of df refere to which price data

    Returns:
    - pd.DataFrame: Processed DataFrame.
    """
    # Step 1: Rename columns
    formated_df = formated_df.rename(columns={
        formated_df_columns[0]: 'date',
        formated_df_columns[1]: 'open',
        formated_df_columns[2]: 'high',
        formated_df_columns[3]: 'low',
        formated_df_columns[4]: 'adj_close',
        formated_df_columns[5]: 'volume'
    })

    # Step 2: Set columns to float64
    formated_df = formated_df.astype({'open': 'float32', 'high': 'float32', 'low': 'float32', 'adj_close': 'float32', 'volume': 'float32'})

    # Step 3: Drop all other columns
    columns_to_keep = ['date', 'open', 'high', 'low', 'adj_close', 'volume']
    formated_df = formated_df[columns_to_keep]

    # Step 4: Set 'date' column to datetime type
    formated_df['date'] = pd.to_datetime(formated_df['date'], format='mixed')

    # Step 5: Set 'date' column as the index
    formated_df.set_index('date', inplace=True)

    return formated_df
    

In [4]:
def sent_df_formating(sent_df, sent_df_columns):
    """
    Preprocess a DataFrame by renaming columns, setting columns to float64,
    dropping unnecessary columns, setting the 'date' column to datetime type,
    and setting the 'date' column as the index.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - define which columns of df refere to which price data

    Returns:
    - pd.DataFrame: Processed DataFrame.
    """
    # Step 1: Rename columns
    sent_df = sent_df.rename(columns={
        sent_df_columns[0]: 'date',
        sent_df_columns[1]: 'score',
        sent_df_columns[2]: 'total',
        sent_df_columns[3]: 'positive',
        sent_df_columns[4]: 'negative'
    })

    # Step 2: Set columns to float64
    sent_df = sent_df.astype({'score': 'float32', 'total': 'float32', 'positive': 'float32', 'negative': 'float32'})

    # Step 3: Set 'date' column to datetime type
    sent_df['date'] = pd.to_datetime(sent_df['date'], format='mixed')

    # Step 4: Set 'date' column as the index
    sent_df.set_index('date', inplace=True)

    # Step 4: Drop Nan rows
    sent_df = sent_df.dropna()
    
    return sent_df

In [50]:
def price_df_formating(formated_df, formated_df_columns):
    """
    Preprocess a DataFrame by renaming columns, setting columns to float64,
    dropping unnecessary columns, setting the 'date' column to datetime type,
    and setting the 'date' column as the index.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - define which columns of df refere to which price data

    Returns:
    - pd.DataFrame: Processed DataFrame.
    """
    # Step 1: Rename columns
    formated_df = formated_df.rename(columns={
        formated_df_columns[0]: 'date',
        formated_df_columns[1]: 'open',
        formated_df_columns[2]: 'adj_close'
    })

    # Step 2: Set columns to float64
    formated_df = formated_df.astype({'open': 'float32', 'adj_close': 'float32'})

    # Step 3: Drop all other columns
    columns_to_keep = ['date', 'open', 'adj_close']
    formated_df = formated_df[columns_to_keep]

    # Step 4: Set 'date' column to datetime type
    formated_df['date'] = pd.to_datetime(formated_df['date'], format='mixed')

    # Step 5: Set 'date' column as the index
    formated_df.set_index('date', inplace=True)

    # Step 6: Drop Nan rows
    price_formated_df = formated_df.dropna()

    return price_formated_df

In [6]:
def labeling_df(labeled_df):
    # Create a new column 'Label' and initialize with 0 (constant)
    labeled_df['label'] = 0
    
    # Label -1 (down) where 'Open' is higher than 'Adj Close'
    labeled_df.loc[labeled_df['open'] > labeled_df['adj_close'], 'label'] = 0

    # Label +1 (up) where 'Open' is lower than 'Adj Close'
    labeled_df.loc[labeled_df['open'] < labeled_df['adj_close'], 'label'] = 1

    return labeled_df

In [7]:
def merge_df(df, sent_df):
    
    # Merge two df on their indexes
    merged_df = pd.merge(df, sent_df, left_index=True, right_index=True)
    
    return merged_df

## 1. Feature Engineering

### A. Moving Average (MA(5) & MA(20))

In [9]:
def moving_averages(df, column_name='adj_close', window_sizes=[5, 20]):
    """
    Add Moving Averages (MA) columns to the DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which moving averages are calculated.
    - window_sizes (list): List of window sizes for moving averages. Default is [5, 20].

    Returns:
    - pd.DataFrame: DataFrame with added MA columns.
    """
    for window_size in window_sizes:
        ma_column_name = f'MA_{window_size}'
        df[ma_column_name] = df[column_name].rolling(window=window_size).mean()

    return df

### B. Bollinger Band (BB up & BB down)

In [10]:
def bollinger_bands(df, column_name='adj_close', window_size=20, num_std_dev=2):
    """
    Calculate Bollinger Bands for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which Bollinger Bands are calculated.
    - window_size (int): Window size for the moving average. Default is 20.
    - num_std_dev (int): Number of standard deviations for the upper and lower bands. Default is 2.

    Returns:
    - pd.DataFrame: DataFrame with added columns for Bollinger Bands (BB up, BB down).
    """
    # Calculate the rolling mean (middle band)
    df['middle_band'] = df[column_name].rolling(window=window_size).mean()

    # Calculate the rolling standard deviation
    df['std_dev'] = df[column_name].rolling(window=window_size).std()

    # Calculate Bollinger Bands
    df['bb_up'] = df['middle_band'] + num_std_dev * df['std_dev']
    df['bb_down'] = df['middle_band'] - num_std_dev * df['std_dev']

    # Drop intermediate columns
    df.drop(['middle_band', 'std_dev'], axis=1, inplace=True)

    return df

### C. Relative Difference in the Percentage of the price (RDP(1))

In [11]:
def rdp(df, column_name='adj_close'):
    """
    Calculate Relative Difference in the Percentage of the price (RDP(1)) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which RDP(1) is calculated.

    Returns:
    - pd.DataFrame: DataFrame with an added column for RDP(1).
    """
    # Calculate RDP(1)
    df['rdp_1'] = df[column_name].pct_change() * 100

    return df

### D. Bias Ratio (BIAS(6), BIAS(12) & BIAS(24))

In [12]:
def bias(df, column_name='adj_close', ma_windows=[6, 12, 24]):
    """
    Calculate Bias Ratios (BIAS) for specified moving average windows for a column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which BIAS is calculated.
    - ma_windows (list): List of moving average window sizes. Default is [6, 12, 24].

    Returns:
    - pd.DataFrame: DataFrame with added columns for BIAS(6), BIAS(12), and BIAS(24).
    """
    for window_size in ma_windows:
        ma_column_name = f'MA_{window_size}'
        bias_column_name = f'BIAS_{window_size}'

        # Calculate the moving average
        df[ma_column_name] = df[column_name].rolling(window=window_size).mean()

        # Calculate BIAS
        df[bias_column_name] = ((df[column_name] - df[ma_column_name]) / df[ma_column_name]) * 100

        # Drop intermediate columns
        df.drop(ma_column_name, axis=1, inplace=True)

    return df

### E. Relative Strength Index (RSI)

In [13]:
def rsi(df, column_name='adj_close', window=14):
    """
    Calculate the Relative Strength Index (RSI) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which RSI is calculated. Default is 'Close'.
    - window (int): Window size for RSI calculation. Default is 14.

    Returns:
    - pd.DataFrame: DataFrame with an added column for RSI.
    """
    # Calculate daily price changes
    df['price_change'] = df[column_name].diff()

    # Calculate the average gain and average loss over the specified window
    df['gain'] = df['price_change'].apply(lambda x: x if x > 0 else 0).rolling(window=window, min_periods=1).mean()
    df['loss'] = -df['price_change'].apply(lambda x: x if x < 0 else 0).rolling(window=window, min_periods=1).mean()

    # Calculate relative strength (RS)
    df['rs'] = df['gain'] / df['loss']

    # Calculate RSI
    df['rsi'] = 100 - (100 / (1 + df['rs']))

    # Drop intermediate columns
    df.drop(['price_change', 'gain', 'loss', 'rs'], axis=1, inplace=True)

    return df

### F. Exponential Moving Average (EMA(12) & EMA(26))

In [14]:
def ema(df, column_name='adj_close', ema_short=12, ema_long=26):
    """
    Calculate Exponential Moving Averages (EMA) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which EMA is calculated. Default is 'Close'.
    - ema_short (int): Short-term EMA window size. Default is 12.
    - ema_long (int): Long-term EMA window size. Default is 26.

    Returns:
    - pd.DataFrame: DataFrame with added columns for EMA(12) and EMA(26).
    """
    # Calculate EMA(12)
    df['ema_12'] = df[column_name].ewm(span=ema_short, adjust=False).mean()

    # Calculate EMA(26)
    df['ema_26'] = df[column_name].ewm(span=ema_long, adjust=False).mean()

    return df

### G. Moving Average Convergence/Divergence (MACD)

In [15]:
def macd(df, column_name='adj_close', ema_short=12, ema_long=26, signal_period=9):
    """
    Calculate Moving Average Convergence Divergence (MACD) and its signal line for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which MACD is calculated. Default is 'Close'.
    - ema_short (int): Short-term EMA window size. Default is 12.
    - ema_long (int): Long-term EMA window size. Default is 26.
    - signal_period (int): Signal line EMA window size. Default is 9.

    Returns:
    - pd.DataFrame: DataFrame with added columns for MACD, Signal Line, and MACD Histogram.
    """
    # Calculate short-term EMA
    df['ema_short'] = df[column_name].ewm(span=ema_short, adjust=False).mean()

    # Calculate long-term EMA
    df['ema_long'] = df[column_name].ewm(span=ema_long, adjust=False).mean()

    # Calculate MACD Line
    df['dif'] = df['ema_short'] - df['ema_long']

    # Calculate Signal Line
    df['signal_line'] = df['dif'].ewm(span=signal_period, adjust=False).mean()

    # Calculate MACD Histogram
    df['osc'] = df['dif'] - df['signal_line']

    # Drop intermediate columns
    df.drop(['ema_short', 'ema_long'], axis=1, inplace=True)

    return df

### H. Psychological Line (PSY(12) & PSY(24))

In [16]:
def psy(df, column_name='adj_close', psy_short=12, psy_long=24):
    """
    Calculate Psychological Line (PSY) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which PSY is calculated. Default is 'Close'.
    - psy_short (int): Short-term PSY window size. Default is 12.
    - psy_long (int): Long-term PSY window size. Default is 24.

    Returns:
    - pd.DataFrame: DataFrame with added columns for PSY(12) and PSY(24).
    """
    # Calculate the percentage of days where the closing price is higher than the previous day's closing price
    df['price_up'] = df[column_name].diff() > 0

    # Calculate PSY(12)
    df['psy_12'] = df['price_up'].rolling(window=psy_short).mean() * 100

    # Calculate PSY(24)
    df['psy_24'] = df['price_up'].rolling(window=psy_long).mean() * 100

    # Drop intermediate columns
    df.drop(['price_up'], axis=1, inplace=True)

    return df

### I. Williams %R (WMS%R)

In [17]:
def williams_percent_r(df, high_column='high', low_column='low', adj_close_column='adj_close', window=14):
    """
    Calculate Williams %R for a specified high, low, and close columns in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - high_column (str): Name of the column containing high prices. Default is 'High'.
    - low_column (str): Name of the column containing low prices. Default is 'Low'.
    - adj_close_column (str): Name of the column containing close prices. Default is 'Close'.
    - window (int): Window size for Williams %R calculation. Default is 14.

    Returns:
    - pd.DataFrame: DataFrame with an added column for Williams %R.
    """
    # Calculate highest high and lowest low over the specified window
    df['hh'] = df[high_column].rolling(window=window).max()
    df['ll'] = df[low_column].rolling(window=window).min()

    # Calculate Williams %R
    df['williams_%r'] = (df['hh'] - df[adj_close_column]) / (df['hh'] - df['ll']) * -100

    # Drop intermediate columns
    df.drop(['hh', 'll'], axis=1, inplace=True)

    return df

### J. Stochastic Oscillator (Stochastic%K & Stochastic%D)

In [18]:
def stochastic_oscillator(df, high_column='high', low_column='low', adj_close_column='adj_close', k_window=14, d_window=3):
    """
    Calculate Stochastic Oscillator (%K and %D) for specified high, low, and close columns in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - high_column (str): Name of the column containing high prices. Default is 'High'.
    - low_column (str): Name of the column containing low prices. Default is 'Low'.
    - close_column (str): Name of the column containing close prices. Default is 'Close'.
    - k_window (int): Window size for %K calculation. Default is 14.
    - d_window (int): Window size for %D calculation. Default is 3.

    Returns:
    - pd.DataFrame: DataFrame with added columns for Stochastic %K and %D.
    """
    # Calculate lowest low and highest high over the specified window
    df['ll'] = df[low_column].rolling(window=k_window).min()
    df['hh'] = df[high_column].rolling(window=k_window).max()

    # Calculate Stochastic %K
    df['stochastic_%k'] = ((df[adj_close_column] - df['ll']) / (df['hh'] - df['ll'])) * 100

    # Calculate Stochastic %D (3-day simple moving average of %K)
    df['stochastic_%d'] = df['stochastic_%k'].rolling(window=d_window).mean()

    # Drop intermediate columns
    df.drop(['ll', 'hh'], axis=1, inplace=True)

    return df

### K. Percentage of Price Change (PROC)

In [19]:
def proc(df, column_name='adj_close', window=1):
    """
    Calculate Percentage of Price Change (PROC) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which PROC is calculated. Default is 'Close'.
    - window (int): Window size for PROC calculation. Default is 1.

    Returns:
    - pd.DataFrame: DataFrame with an added column for PROC.
    """
    # Calculate the percentage change in price using rolling window
    df['proc'] = df[column_name].pct_change().rolling(window=window).mean() * 100

    return df

### L. Momentum (MO(1))

In [20]:
def momentum(df, column_name='adj_close', window=1):
    """
    Calculate Momentum (MO) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which Momentum is calculated. Default is 'Close'.
    - window (int): Window size for Momentum calculation. Default is 1.

    Returns:
    - pd.DataFrame: DataFrame with an added column for Momentum.
    """
    # Calculate the difference in price over the specified window
    df['momentum'] = df[column_name].diff(window)

    return df

### M. First-Order Lag (LAG(1))

In [21]:
def first_order_lag(df, column_name='adj_close', lag=1):
    """
    Calculate First-Order Lag (LAG(1)) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which the lag is calculated. Default is 'Close'.
    - lag (int): Number of periods to lag. Default is 1.

    Returns:
    - pd.DataFrame: DataFrame with an added column for the First-Order Lag.
    """
    # Calculate the First-Order Lag using the shift() method
    df[f'LAG_{lag}'] = df[column_name].shift(lag)

    return df

### N. Trading Volume (VOL)

In [22]:
def trading_volume(df, volume_column='volume'):
    """
    Calculate Trading Volume (VOL) for a specified column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - volume_column (str): Name of the column containing trading volume. Default is 'Volume'.

    Returns:
    - pd.DataFrame: DataFrame with an added column for Trading Volume.
    """
    df['vol'] = df[volume_column]

    return df

## 2. Preprocessing

### a. Removing columns and rows

In [23]:
def drop_columns(df, columns_to_drop=['open', 'high', 'low', 'adj_close', 'volume']):
    """
    Drop specified columns from a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - columns_to_drop (list): List of column names to drop. Default is ['Open', 'High', 'Low', 'Adj_Close', 'Volume'].

    Returns:
    - pd.DataFrame: DataFrame with specified columns dropped.
    """
    # Drop specified columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    return df

In [24]:
def drop_rows(df):
    """
    Drop all rows with NaN values from a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.

    Returns:
    - pd.DataFrame: DataFrame with NaN rows dropped.
    - list: List of indices corresponding to dropped rows.
    """
    # Drop rows with NaN values
    cleaned_df = df.dropna()

    return cleaned_df

### b. Scaling

In [25]:
def scale_dataframe(df):
    """
    Scale a DataFrame using Standard scaling.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.

    Returns:
    - pd.DataFrame: Scaled DataFrame.
    """
    # Scale the selected columns
    scaler = StandardScaler()

    index_column = df.index
    
    date_column = df['label']
    int_df = df.drop(columns=['label'])
    
    columns_to_scale = int_df.columns
    
    scaled_df = pd.DataFrame(scaler.fit_transform(int_df), columns=columns_to_scale)
    scaled_df.index = index_column
    scaled_df['label'] = date_column
    
    return scaled_df

### c. Train Test Split

In [26]:
def train_test_split(df, test_size=0.2):
    """
    Split a time series dataset into training, testing, and validation sets.

    Parameters:
    - df: NumPy array or matrix, the input time series dataset.
    - test_size: Float, the proportion of the dataset to include in the test split.
    - val_size: Float, the proportion of the dataset to include in the validation split.

    Returns:
    - df_train, df_test: Pandas arrays, representing features and target values for each set.
    """
    
    # Extract index number of splitting points
    len_df = len(df)
    index_1 = round(len_df*(1-(test_size)))
    index_2 = index_1 +1

    # Extract values at previously calculated splitting points
    date_1 = df.index[index_1]
    date_2 = df.index[index_2]

    # Construct train_df, val_df and test_df
    df_train = df[:date_1]
    df_test = df[date_2:]
    
    return df_train, df_test

### d. Reshape Matrix

In [27]:
def input_matrix_split_X_y(df, window_size=5):
    """
    Reshape a DataFrame into a 3D NumPy arrays (num_observations, window_size, num_features)

    Parameters:
    - df: DataFrame with a list of time series data
    - sequence_length: the number of time steps to consider for each observation

    Returns:
    - X, y: a 3D NumPy arrays, one for the features and one for the lables
    """
    df_np = df.to_numpy()
    X = []
    y = []
    
    df_X = df.drop('label', axis=1)
    df_y = df['label']

    for i in range(len(df_np)-(window_size)):
        row = df_X[i:i+window_size]
        X.append(row)
        label = df_y[i+(window_size)]
        y.append(label)

    X = np.array(X)
    y = np.array(y)
    y = np.expand_dims(y, axis=-1)
    
    return X, y

## 3. Model Iterations

### Baseline Model

In [28]:
def baseline_model(X_train, y_train, X_test, y_test, window_size=5, optimizer_name='adam', loss_function='binary_crossentropy', metrics_list=['accuracy'], patience=5, validation_split=0.2, batch_size=64, epochs=100, verbose=1):
    
    #############################
    #  1 - Model architecture   #
    ############################# 
    
    # [1, 5] layers
    # nodes per layer [30, 70]
    # activation function: relu
    # optimizer: Adam
    # learning rate = [0.001, 0.1]
    
    model = models.Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(window_size, X_train.shape[-1])))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    
    # $CHALLENGIFY_END
    
    #############################
    #  2 - Optimization Method  #
    #############################
    model.compile(loss= loss_function,
                  optimizer = optimizer_name, 
                  metrics = metrics_list) 

    #############################
    #  3 - Training Method  #
    #############################
    es = EarlyStopping(patience=patience, restore_best_weights=True)
    
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs, 
                        callbacks=[es],
                        verbose=verbose)
    #############################
    #  4 - Validation Method  #
    #############################
    model_acc = model.evaluate(X_test, y_test, verbose=0)

    return model, history, model_acc

### a. LSTM Model

In [66]:
def lstm_model(X_train, y_train, X_test, y_test, window_size=5, optimizer_name='adam', loss_function='binary_crossentropy', metrics_list=['accuracy'], patience=5, validation_split=0.2, batch_size=64, epochs=100, verbose=1):
    
    #############################
    #  1 - Model architecture   #
    ############################# 
    
    # [1, 5] layers
    # nodes per layer [30, 70]
    # activation function: relu
    # optimizer: Adam
    # learning rate = [0.001, 0.1]
    
    model = models.Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(window_size, X_train.shape[-1])))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    
    # $CHALLENGIFY_END
    
    #############################
    #  2 - Optimization Method  #
    #############################
    model.compile(loss= loss_function,
                  optimizer = optimizer_name, 
                  metrics = metrics_list) 

    #############################
    #  3 - Training Method  #
    #############################
    es = EarlyStopping(patience=patience, restore_best_weights=True)
    
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs, 
                        callbacks=[es],
                        verbose=verbose)
    #############################
    #  4 - Validation Method  #
    #############################
    model_acc = model.evaluate(X_test, y_test, verbose=0)

    return model, history, model_acc

### b. GRU Model

In [67]:
def gru_model(X_train, y_train, X_test, y_test, window_size=5, optimizer_name='adam', loss_function='binary_crossentropy', metrics_list=['accuracy'], patience=5, validation_split=0.2, batch_size=64, epochs=100, verbose=1):
    
    #############################
    #  1 - Model architecture   #
    ############################# 
    
    # [1, 5] layers
    # nodes per layer [30, 70]
    # activation function: relu
    # optimizer: Adam
    # learning rate = [0.001, 0.1]
    
    model = models.Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(window_size, X_train.shape[-1])))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    
    # $CHALLENGIFY_END
    
    #############################
    #  2 - Optimization Method  #
    #############################
    model.compile(loss= loss_function,
                  optimizer = optimizer_name, 
                  metrics = metrics_list) 

    #############################
    #  3 - Training Method  #
    #############################
    es = EarlyStopping(patience=patience, restore_best_weights=True)
    
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs, 
                        callbacks=[es],
                        verbose=verbose)
    #############################
    #  4 - Validation Method  #
    #############################
    model_acc = model.evaluate(X_test, y_test, verbose=0)

    return model, history, model_acc

### c. Transformer Model

In [68]:
def transformer_model(X_train, y_train, X_test, y_test, window_size=5, optimizer_name='adam', loss_function='binary_crossentropy', metrics_list=['accuracy'], patience=5, validation_split=0.2, batch_size=64, epochs=100, verbose=1):
    
    #############################
    #  1 - Model architecture   #
    ############################# 
    
    # [1, 5] layers
    # nodes per layer [30, 70]
    # activation function: relu
    # optimizer: Adam
    # learning rate = [0.001, 0.1]
    
    model = models.Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(window_size, X_train.shape[-1])))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    
    # $CHALLENGIFY_END
    
    #############################
    #  2 - Optimization Method  #
    #############################
    model.compile(loss= loss_function,
                  optimizer = optimizer_name, 
                  metrics = metrics_list) 

    #############################
    #  3 - Training Method  #
    #############################
    es = EarlyStopping(patience=patience, restore_best_weights=True)
    
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs, 
                        callbacks=[es],
                        verbose=verbose)
    #############################
    #  4 - Validation Method  #
    #############################
    model_acc = model.evaluate(X_test, y_test, verbose=0)

    return model, history, model_acc

### d. LSTM Bidirectional Model

In [69]:
def bi_lstm_model(X_train, y_train, X_test, y_test, window_size=5, optimizer_name='adam', loss_function='binary_crossentropy', metrics_list=['accuracy'], patience=5, validation_split=0.2, batch_size=64, epochs=100, verbose=1):
    
    #############################
    #  1 - Model architecture   #
    ############################# 
    
    # [1, 5] layers
    # nodes per layer [30, 70]
    # activation function: relu
    # optimizer: Adam
    # learning rate = [0.001, 0.1]
    
    model = models.Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(window_size, X_train.shape[-1])))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    
    # $CHALLENGIFY_END
    
    #############################
    #  2 - Optimization Method  #
    #############################
    model.compile(loss= loss_function,
                  optimizer = optimizer_name, 
                  metrics = metrics_list) 

    #############################
    #  3 - Training Method  #
    #############################
    es = EarlyStopping(patience=patience, restore_best_weights=True)
    
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs, 
                        callbacks=[es],
                        verbose=verbose)
    #############################
    #  4 - Validation Method  #
    #############################
    model_acc = model.evaluate(X_test, y_test, verbose=0)

    return model, history, model_acc

## 4. Model Iterations Visuals

### a. df including only prices

In [82]:
def pipeline_basline_model(path, formated_df_columns_price, columns_to_drop):
    #############################
    #  1 - Importing Data       #
    #############################
    df = import_data(path)
    
    #############################
    #  2 - Basic Formating      #
    #############################
    formated_df = price_df_formating(df, formated_df_columns_price)
    labeled_formated_df = labeling_df(formated_df)
    
    #############################
    #  4 - Preprocessing        #
    #############################
    clean_merged_df = drop_columns(labeled_formated_df, columns_to_drop)
    clean_merged_df = drop_rows(clean_merged_df)
    scaled_clean_merged_df = scale_dataframe(clean_merged_df)
    split_scaled_clean_merged_df = train_test_split(scaled_clean_merged_df)
    X_train, y_train = input_matrix_split_X_y(split_scaled_clean_merged_df[0])
    X_test, y_test = input_matrix_split_X_y(split_scaled_clean_merged_df[1])
    
    #############################
    #  5 - Models               #
    #############################
    baseline_model1 = baseline_model(X_train, y_train, X_test, y_test)
    
    return baseline_model1

### b. df including only features

In [78]:
def pipeline_features_df(path, formated_df_columns):

    #############################
    #  1 - Importing Data       #
    #############################
    df = import_data(path)
    
    #############################
    #  2 - Basic Formating      #
    #############################
    formated_df = df_formating(df, formated_df_columns)
    labeled_formated_df = labeling_df(formated_df)
    
    #############################
    #  3 - Feature Engineering  #
    #############################
    moving_averages(labeled_formated_df)
    bollinger_bands(labeled_formated_df)
    rdp(labeled_formated_df)
    bias(labeled_formated_df)
    rsi(labeled_formated_df)
    ema(labeled_formated_df)
    macd(labeled_formated_df)
    psy(labeled_formated_df)
    williams_percent_r(labeled_formated_df)
    stochastic_oscillator(labeled_formated_df)
    proc(labeled_formated_df)
    momentum(labeled_formated_df)
    first_order_lag(labeled_formated_df)
    trading_volume(labeled_formated_df)
    
    #############################
    #  4 - Preprocessing        #
    #############################
    clean_merged_df = drop_columns(labeled_formated_df)
    clean_merged_df = drop_rows(clean_merged_df)
    scaled_clean_merged_df = scale_dataframe(clean_merged_df)
    split_scaled_clean_merged_df = train_test_split(scaled_clean_merged_df)
    X_train, y_train = input_matrix_split_X_y(split_scaled_clean_merged_df[0])
    X_test, y_test = input_matrix_split_X_y(split_scaled_clean_merged_df[1])
    
    #############################
    #  5 - Models               #
    #############################
    f_lstm_model = lstm_model(X_train, y_train, X_test, y_test)
    f_gru_model = gru_model(X_train, y_train, X_test, y_test)
    f_transformer_model = transformer_model(X_train, y_train, X_test, y_test)
    f_bi_lstm_model = bi_lstm_model(X_train, y_train, X_test, y_test)
    
    return f_lstm_model, f_gru_model, f_transformer_model, f_bi_lstm_model

### c. df including features and sentimental analysis

In [77]:
def pipeline_features_sent_df(path, formated_df_columns, sent_df_columns): 

    #############################
    #  1 - Importing Data       #
    #############################
    df = import_data(path)
    sent_df = import_data_sent(path, sent_df_columns)
    
    #############################
    #  2 - Basic Formating      #
    #############################
    formated_df = df_formating(df, formated_df_columns)
    formated_sent_df = sent_df_formating(sent_df, sent_df_columns)
    labeled_formated_df = labeling_df(formated_df)
    merged_df = merge_df(labeled_formated_df, formated_sent_df)
    
    #############################
    #  3 - Feature Engineering  #
    #############################
    moving_averages(merged_df)
    bollinger_bands(merged_df)
    rdp(merged_df)
    bias(merged_df)
    rsi(merged_df)
    ema(merged_df)
    macd(merged_df)
    psy(merged_df)
    williams_percent_r(merged_df)
    stochastic_oscillator(merged_df)
    proc(merged_df)
    momentum(merged_df)
    first_order_lag(merged_df)
    trading_volume(merged_df)
    
    #############################
    #  4 - Preprocessing        #
    #############################
    clean_merged_df = drop_columns(merged_df)
    clean_merged_df = drop_rows(clean_merged_df)
    scaled_clean_merged_df = scale_dataframe(clean_merged_df)
    split_scaled_clean_merged_df = train_test_split(scaled_clean_merged_df)
    X_train, y_train = input_matrix_split_X_y(split_scaled_clean_merged_df[0])
    X_test, y_test = input_matrix_split_X_y(split_scaled_clean_merged_df[1])
    
    #############################
    #  5 - Models               #
    #############################
    f_sent_lstm_model = lstm_model(X_train, y_train, X_test, y_test)
    f_sent_gru_model = gru_model(X_train, y_train, X_test, y_test)
    f_sent_transformer_model = transformer_model(X_train, y_train, X_test, y_test)
    f_sent_bi_lstm_model = bi_lstm_model(X_train, y_train, X_test, y_test)
    
    return f_sent_lstm_model, f_sent_gru_model, f_sent_transformer_model, f_sent_bi_lstm_model

### d. Visualisation functions

In [85]:
path = "cleaned_merged_data-sample-with-sentiment-v02.csv"
formated_df_columns = ['Datetime', 'Open-TSLA', 'High-TSLA', 'Low-TSLA', 'Adj Close-TSLA', 'Volume-TSLA']
formated_df_columns_price = ['Datetime','Open-TSLA', 'Adj Close-TSLA']
sent_df_columns = ['Datetime', 'score_int', 'total_tweets', 'share_of_positive', 'share_of_negative']
columns_to_drop = ['open']

base_model, base_history, base_model_acc = pipeline_basline_model(path, formated_df_columns_price, columns_to_drop)


f_lstm_model, f_gru_model, f_transformer_model, f_bi_lstm_model = pipeline_features_df(path, formated_df_columns)
f_lstm_model, f_lstm_history, f_lstm_model_acc = f_lstm_model
f_gru_model, f_gru_history, f_gru_model_acc = f_gru_model
f_trans_model, f_trans_history, f_trans_model_acc = f_transformer_model
f_bi_lstm_model, f_bi_lstm_history, f_bi_lstm_model_acc = f_bi_lstm_model


f_sent_lstm_model, f_sent_gru_model, f_sent_transformer_model, f_sent_bi_lstm_model = pipeline_features_sent_df(path, formated_df_columns, sent_df_columns)
f_sent_lstm_model, f_sent_lstm_history, f_sent_lstm_model_acc = f_sent_lstm_model
f_sent_gru_model, f_sent_gru_history, f_sent_gru_model_acc = f_sent_gru_model
f_sent_trans_model, f_sent_trans_history, f_sent_trans_model_acc = f_sent_transformer_model
f_sent_bi_lstm_model, f_sent_bi_lstm_history, f_sent_bi_lstm_model_acc = f_sent_bi_lstm_model

Epoch 1/100


/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]
/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]


13/13 [==============================] - 1s 17ms/step - loss: 0.6942 - accuracy: 0.5121 - val_loss: 0.6937 - val_accuracy: 0.4915
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6916 - accuracy: 0.5395 - val_loss: 0.6937 - val_accuracy: 0.4915
Epoch 3/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6902 - accuracy: 0.5460 - val_loss: 0.6937 - val_accuracy: 0.4915
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6905 - accuracy: 0.5440 - val_loss: 0.6944 - val_accuracy: 0.4836
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6909 - accuracy: 0.5405 - val_loss: 0.6952 - val_accuracy: 0.4706
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 0.6902 - accuracy: 0.5400 - val_loss: 0.6957 - val_accuracy: 0.4726
Epoch 7/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6902 - accuracy: 0.5417 - val_loss: 0.6944 - val_accuracy: 0.4866
Epoch 8/100
13/13 [=====

/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]
/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]


13/13 [==============================] - 1s 17ms/step - loss: 0.6978 - accuracy: 0.4947 - val_loss: 0.6942 - val_accuracy: 0.5121
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.5221 - val_loss: 0.6957 - val_accuracy: 0.4980
Epoch 3/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.5261 - val_loss: 0.6981 - val_accuracy: 0.4939
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6906 - accuracy: 0.5299 - val_loss: 0.6991 - val_accuracy: 0.4455
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6877 - accuracy: 0.5437 - val_loss: 0.6979 - val_accuracy: 0.4929
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6875 - accuracy: 0.5449 - val_loss: 0.7004 - val_accuracy: 0.4414
Epoch 1/100
13/13 [==============================] - 1s 17ms/step - loss: 0.6989 - accuracy: 0.4975 - val_loss: 0.6923 - val_accuracy: 0.5343
Epoch 2/100
13/13 [====

/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]
/var/folders/l4/q_79lrcx3ps_z7hltvr9nl4c0000gn/T/ipykernel_75944/241684724.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = df_y[i+(window_size)]


13/13 [==============================] - 1s 16ms/step - loss: 0.7013 - accuracy: 0.4868 - val_loss: 0.6947 - val_accuracy: 0.5202
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6907 - accuracy: 0.5315 - val_loss: 0.6956 - val_accuracy: 0.5182
Epoch 3/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6880 - accuracy: 0.5391 - val_loss: 0.6960 - val_accuracy: 0.4990
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6855 - accuracy: 0.5505 - val_loss: 0.6994 - val_accuracy: 0.4596
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6825 - accuracy: 0.5599 - val_loss: 0.7037 - val_accuracy: 0.4525
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 0.6807 - accuracy: 0.5617 - val_loss: 0.7005 - val_accuracy: 0.4838
Epoch 1/100
13/13 [==============================] - 1s 17ms/step - loss: 0.7021 - accuracy: 0.4914 - val_loss: 0.6950 - val_accuracy: 0.4980
Epoch 2/100
13/13 [====

In [ ]:
model.summary()
history = train_model()
print(f'Model Accuracy {model_acc[1]:.4f}')

def plot_loss_accuracy(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

plot_loss_accuracy(history)

## 5 Model Optimisation template idea

### a. LSTM

In [54]:
%time

# Model hyperparameters
num_layers = [1, 2, 3] # 2, 2, 2, 2
layer_sizes = [50, 60, 70, 80] # 50, 60, 80, 70
batch_sizes = [64, 128] # 128, 64, 64, 128
learning_rates = [0.1, 1.0, 2.0] # 0.1, 1, 2, 0.1
epochs = [50, 60, 70, 80] # 50, 60, 50, 80
activ_func = 'relu'

best_model = None
best_performance = 0.0
best_weights = 0.0
best_hyperparameters = {}
count = 0

# Iterate over hyperparameter combinations
for num_layer in num_layers:
    for layer_size in layer_sizes:
        for batch_size in batch_sizes:
            for learning_rate in learning_rates:
                for epoch in epochs:
                    count += 1
                    # Build the LSTM model
                    model = Sequential()
                    
                    # Add the specified number of LSTM layers
                    for _ in range(num_layer):
                        model.add(LSTM(layer_size, activation=activ_func, input_shape=(5, X_train.shape[-1]), return_sequences=True))
                    
                    # Flatten if there are multiple LSTM layers
                    if num_layer > 1:
                        model.add(Flatten())
                    
                    # Output layer
                    model.add(Dense(1, activation='sigmoid'))
                    
                    # Compile the model
                    optimizer = Adam(learning_rate=learning_rate)
                    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
                    
                    # Print model summary
                    print(model.summary())
                    
                    # Train the model
                    es = EarlyStopping(patience=5, restore_best_weights=True)
                    history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, validation_split=0.2, callbacks=[es], verbose=2)
                    
                    val_performance = history.history['val_accuracy'][-1]
                    
                    # Check if this configuration is the best so far
                    if val_performance > best_performance:
                        best_performance = val_performance
                        best_model = model
                        best_hyperparameters = {
                            'num_layer': num_layer,
                            'layer_size': layer_size,
                            'batch_size': batch_size,
                            'learning_rate': learning_rate,
                            'epochs': epoch
                        }

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.11 µs
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 50)             15600     
                                                                 
 dense_2 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 15651 (61.14 KB)
Trainable params: 15651 (61.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/50
13/13 - 1s - loss: 1.0806 - accuracy: 0.5213 - val_loss: 0.7726 - val_accuracy: 0.5192 - 541ms/epoch - 42ms/step
Epoch 2/50
13/13 - 0s - loss: 0.7574 - accuracy: 0.5038 - val_loss: 0.8236 - val_accuracy: 0.4727 - 37ms/epoch - 3ms/step
Epoch 3/50
13/13 - 0s - loss: 0.7194 - accuracy: 0.5574 - val_loss: 0.8042 - val_accuracy: 0.4646 -

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 5, 50)             15600     
                                                                 
 dense_3 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 15651 (61.14 KB)
Trainable params: 15651 (61.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/60
13/13 - 1s - loss: 1.1214 - accuracy: 0.4985 - val_loss: 0.8895 - val_accuracy: 0.5152 - 564ms/epoch - 43ms/step
Epoch 2/60
13/13 - 0s - loss: 0.7442 - accuracy: 0.5178 - val_loss: 1.0395 - val_accuracy: 0.4768 - 35ms/epoch - 3ms/step
Epoch 3/60
13/13 - 0s - loss: 0.7109 - accuracy: 0.5467 - val_loss: 0.8592 - val_accuracy: 0.4919 - 34ms/epoch - 3ms/step
Epoch 4/60
13/13 - 0s - loss: 0.6868 - ac

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 5, 50)             15600     
                                                                 
 dense_4 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 15651 (61.14 KB)
Trainable params: 15651 (61.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/70
13/13 - 1s - loss: 1.1236 - accuracy: 0.4929 - val_loss: 0.7779 - val_accuracy: 0.4970 - 549ms/epoch - 42ms/step
Epoch 2/70
13/13 - 0s - loss: 0.7476 - accuracy: 0.5259 - val_loss: 1.0313 - val_accuracy: 0.4747 - 36ms/epoch - 3ms/step
Epoch 3/70
13/13 - 0s - loss: 0.7107 - accuracy: 0.5297 - val_loss: 0.7116 - val_accuracy: 0.4970 - 35ms/epoch - 3ms/step
Epoch 4/70
13/13 - 0s - loss: 0.6905 - ac

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 5, 50)             15600     
                                                                 
 dense_5 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 15651 (61.14 KB)
Trainable params: 15651 (61.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/80
13/13 - 1s - loss: 1.1461 - accuracy: 0.5173 - val_loss: 0.7727 - val_accuracy: 0.4616 - 644ms/epoch - 50ms/step
Epoch 2/80
13/13 - 0s - loss: 0.7217 - accuracy: 0.5183 - val_loss: 0.7462 - val_accuracy: 0.5091 - 36ms/epoch - 3ms/step
Epoch 3/80
13/13 - 0s - loss: 0.7009 - accuracy: 0.5287 - val_loss: 0.7590 - val_accuracy: 0.4758 - 36ms/epoch - 3ms/step
Epoch 4/80
13/13 - 0s - loss: 0.6815 - ac

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 5, 50)             15600     
                                                                 
 dense_6 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 15651 (61.14 KB)
Trainable params: 15651 (61.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/50
13/13 - 1s - loss: 7.1872 - accuracy: 0.4934 - val_loss: 7.9262 - val_accuracy: 0.4838 - 537ms/epoch - 41ms/step
Epoch 2/50
13/13 - 0s - loss: 7.4225 - accuracy: 0.5147 - val_loss: 7.7022 - val_accuracy: 0.4970 - 35ms/epoch - 3ms/step
Epoch 3/50
13/13 - 0s - loss: 7.6468 - accuracy: 0.4992 - val_loss: 7.7363 - val_accuracy: 0.4939 - 37ms/epoch - 3ms/step
Epoch 4/50
13/13 - 0s - loss: 7.2459 - ac

...

Model: "sequential_92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_91 (LSTM)              (None, 5, 80)             34560     
                                                                 
 dense_91 (Dense)            (None, 5, 1)              81        
                                                                 
Total params: 34641 (135.32 KB)
Trainable params: 34641 (135.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/60
7/7 - 1s - loss: 6.2322 - accuracy: 0.5249 - val_loss: 8.0108 - val_accuracy: 0.4798 - 546ms/epoch - 78ms/step
Epoch 2/60
7/7 - 0s - loss: 7.3837 - accuracy: 0.5206 - val_loss: 7.9339 - val_accuracy: 0.4848 - 34ms/epoch - 5ms/step
Epoch 3/60
7/7 - 0s - loss: 7.4778 - accuracy: 0.5145 - val_loss: 7.9341 - val_accuracy: 0.4848 - 33ms/epoch - 5ms/step
Epoch 4/60
7/7 - 0s - loss: 7.3852 - accurac

Model: "sequential_93"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_92 (LSTM)              (None, 5, 80)             34560     
                                                                 
 dense_92 (Dense)            (None, 5, 1)              81        
                                                                 
Total params: 34641 (135.32 KB)
Trainable params: 34641 (135.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/70
7/7 - 1s - loss: 6.1711 - accuracy: 0.5211 - val_loss: 8.1799 - val_accuracy: 0.4697 - 551ms/epoch - 79ms/step
Epoch 2/70
7/7 - 0s - loss: 7.2298 - accuracy: 0.5310 - val_loss: 8.1799 - val_accuracy: 0.4697 - 31ms/epoch - 4ms/step
Epoch 3/70
7/7 - 0s - loss: 7.2186 - accuracy: 0.5320 - val_loss: 8.1799 - val_accuracy: 0.4697 - 33ms/epoch - 5ms/step
Epoch 4/70
7/7 - 0s - loss: 7.2438 - accurac

Model: "sequential_94"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_93 (LSTM)              (None, 5, 80)             34560     
                                                                 
 dense_93 (Dense)            (None, 5, 1)              81        
                                                                 
Total params: 34641 (135.32 KB)
Trainable params: 34641 (135.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/80
7/7 - 1s - loss: 7.0406 - accuracy: 0.4632 - val_loss: 7.1625 - val_accuracy: 0.5303 - 563ms/epoch - 80ms/step
Epoch 2/80
7/7 - 0s - loss: 8.1084 - accuracy: 0.4683 - val_loss: 7.1625 - val_accuracy: 0.5303 - 32ms/epoch - 5ms/step
Epoch 3/80
7/7 - 0s - loss: 8.1084 - accuracy: 0.4683 - val_loss: 7.1625 - val_accuracy: 0.5303 - 32ms/epoch - 5ms/step
Epoch 4/80
7/7 - 0s - loss: 8.1084 - accurac

Model: "sequential_95"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_94 (LSTM)              (None, 5, 80)             34560     
                                                                 
 dense_94 (Dense)            (None, 5, 1)              81        
                                                                 
Total params: 34641 (135.32 KB)
Trainable params: 34641 (135.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/50


KeyboardInterrupt: 

In [55]:
best_model.summary()
print(f'Best Model Accuracy: {best_performance}')
print(f'Best Model Hyperparameters: {best_hyperparameters}')
print(f'Number of Model Iterations: {count}')

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_35 (LSTM)              (None, 5, 60)             21120     
                                                                 
 dense_35 (Dense)            (None, 5, 1)              61        
                                                                 
Total params: 21181 (82.74 KB)
Trainable params: 21181 (82.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Best Model Accuracy: 0.5363636016845703
Best Model Hyperparameters: {'num_layer': 1, 'layer_size': 60, 'batch_size': 64, 'learning_rate': 2.0, 'epochs': 60}
Number of Model Iterations: 93


In [56]:
model_acc = best_model.evaluate(X_test, y_test, verbose=1)

print(f'Model Accuracy {model_acc[1]:.4f}')

8/8 [==============================] - 0s 1ms/step - loss: 7.0749 - accuracy: 0.5378
Model Accuracy 0.5378


### b. Graph